In this notebook, we will try to predict if the customers will terminate their credit card services.

We'll be using data from the [Credit Card Customers Dataset](https://www.kaggle.com/sakshigoyal7/credit-card-customers). Each column is precisely described in this link

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier

from sklearn.metrics import accuracy_score, confusion_matrix, precision_recall_fscore_support
from sklearn.model_selection import GridSearchCV, cross_val_score

In [ ]:
full_data = pd.read_csv('../input/credit-card-customers/BankChurners.csv')
full_data

Our target column is the "Attrition_Flag" column.

The data description suggests dropping the last 2 columns, so we will do that

In [ ]:
full_data.drop(
    ['Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1',
     'Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2'],
               axis='columns',
              inplace=True)

In [ ]:
full_data.head()

In [ ]:
full_data['Attrition_Flag'].value_counts()

As we can see, we don't have a lot of attrited target rows, so we will have to ensure that they are evenly distributed during train and test split.

In [ ]:
display(full_data.info())
full_data.describe()

We can see that there are no missing values in this dataset.

We can drop the CLIENTNUM column as the values are indexes of the customers.

Also, the values will definitely require scaling

In [ ]:
full_data.drop(['CLIENTNUM'], axis='columns', inplace=True)

We will also convert the target column to 0 and 1 values.

In [ ]:
full_data['Attrition_Flag'] = pd.get_dummies(full_data['Attrition_Flag'])

To split the data, we will take 80% of the whole dataset as trainig data. However, we will take 80% from both groups of clients who left and the clients who stayed.

In [ ]:
train_test_split = 0.8

clients_remaining = full_data[full_data['Attrition_Flag'] == 1]
clients_left = full_data.drop(clients_remaining.index)

train_data = clients_remaining.sample(frac=train_test_split).append(clients_left.sample(frac=train_test_split))
test_data = full_data.drop(train_data.index)
train_data

# EDA

In [ ]:
categorical_features = ['Gender', 'Dependent_count', 'Education_Level', 'Marital_Status', 'Income_Category',
                       'Card_Category']
train_data.nunique()

In [ ]:
train_data.hist(figsize=(20,20))

We can see that a couple columns like Customer_Age have a distribution quiet close to a normal distribution.

In [ ]:
correlation_matrix = train_data.corr()
plt.figure(figsize=(13,13))
sns.heatmap(correlation_matrix, annot=True)

We can see a couple very strong correlations:
 - Months_On_Book - Customer_Age
 - Avg_Open_To_Buy - Credit_Limit seems like it's practically identical
 - Total_Trans_Ct - Total_Trans_Amt
 - Avg_Utilization_Ratio - Total_Revolving_Bal

# Preprocessing

1. Splitting data into X_train and y_train

In [ ]:
y_train = train_data['Attrition_Flag']
X_train = train_data.drop(['Attrition_Flag'], axis='columns')

y_test = test_data['Attrition_Flag']
X_test = test_data.drop(['Attrition_Flag'], axis='columns')

1. One Hot Encoding

In [ ]:
X_train = pd.get_dummies(X_train)
X_test = pd.get_dummies(X_test)

2. Scaling the data

In [ ]:
scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Training the model

In [ ]:
model_lr = LogisticRegression()
model_rfc = RandomForestClassifier()
model_mlp = MLPClassifier()

Before we compare the models, it would be good to actually look for some better parameters for the MPLClassifier, because the score is very dependant on the architecture of the neural network.

In [ ]:
model_mlp.set_params(max_iter = 50)

mlp_params = {
    'hidden_layer_sizes': [(8), (8, 8), (16), (16, 16), (8, 8, 8)],
    'activation': ['relu', 'logistic'],
    
}

grid_search = GridSearchCV(model_mlp, mlp_params, cv=4)
grid_search.fit(X_train, y_train)

means = grid_search.cv_results_['mean_test_score']
stds = grid_search.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, grid_search.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))

Now that we know the best architecture for the neural network, we can set these params for our model and get to comparing different models

In [ ]:
model_mlp.set_params(**grid_search.best_params_)
model_mlp.set_params(max_iter=200)

In [ ]:
models = [model_lr, model_rfc, model_mlp]

for model in models:
    print(f"{model.__class__.__name__}:")
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)
    print(f"Accuracy: {accuracy_score(y_test, predictions)}")

It seems like the RandomForestClassifier is the most promising choice, so we will train it further and use the grid search method with cross-validation to find the best paramters for it

In [ ]:
rfc_params = {
    'n_estimators': [40, 80, 100, 150, 200],
}

grid_search = GridSearchCV(model_rfc, rfc_params, cv=5)
grid_search.fit(X_train, y_train)

means = grid_search.cv_results_['mean_test_score']
stds = grid_search.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, grid_search.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))

Ok, so now all that remains is to train the model on the best parameters and get the final result on our test set

In [ ]:
model_rfc.set_params(**grid_search.best_params_)
model_rfc.fit(X_train, y_train)
predictions = model_rfc.predict(X_test)
print(f"Accuracy score: {accuracy_score(y_test, predictions)}")

Let's also see the confusion matrix and precision, recall and f1 score

In [ ]:
print(f"Accuracy score: {accuracy_score(y_test, predictions)}")
print()

matrix = confusion_matrix(y_test, predictions)
print('Confusion matrix:')
print(matrix)
print()

precision, recall, f_score, support = precision_recall_fscore_support(y_test, predictions)
print("For each class:")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F score: {f_score}")
print(f"Support: {support}")